# Simple RSX oscillator market orders strategy with manual interaction

In [1]:
import numpy as np
import pandas as pd
from simple.plotly import interactFigure
from simple.jurik import JRSX, JMA
from simple.backtest import getLong, getShort, npBacktestMarket, getProfit
from numba import njit

In [2]:
S = np.load('data/ETHUSDT.2022-05-08.bidask.npz')['ETHUSDT'].view(np.recarray)
len(S)

600964

In [3]:
@njit
def getPos(OSC, Threshold = 10):
    Result = np.zeros_like(OSC)
    p = 0
    for i in range(1, len(OSC)):
        if OSC[i] > Threshold and OSC[i] < OSC[i-1]:
            p = 1
        elif OSC[i] < -Threshold and OSC[i] > OSC[i-1]:
            p = -1
        Result[i] = p
    return Result

In [10]:
def model(Period: int = (50, 20000, 50), Threshold: float = (0, 10)):
    if Period == 0: return 0, {}
    Tick = (S.A + S.B) / 2
    OSC = JRSX(Tick, Period) - 50
    Signal = -getPos(OSC, Threshold)
    maOSC = JMA(OSC, Period*4)

    trades = npBacktestMarket(S.DT, S.A, S.B, Signal, 0)
    Buy, Sell = getLong(trades), getShort(trades)
    P = getProfit(trades)
    Profit = {'x': P.Index, 'y': P.RawPnL.cumsum()}

    return P.Profit.sum(), locals()

In [11]:
interactFigure(model, rows=2,
    Tick={'color': 'gray', 'opacity': 0.3},
    OSC={'color': 'orange', 'opacity': 0.5, 'row': 2},
    maOSC={'color': 'green', 'opacity': 0.5, 'row': 2},
    Signal={'color': 'red', 'opacity': 0.5, 'row': 2, 'secondary_y': True},
    Profit={'mode': 'lines', 'color': 'gray', 'width': 3, 'opacity': 0.5, 'secondary_y': True, 'shape': 'hv', 'connectgaps': True},
    Buy={'mode': 'markers', 'color': 'green', 'symbol': 'triangle-up', 'size': 10, 'line': {'color': 'darkgreen', 'width': 1}},
    Sell={'mode': 'markers', 'color': 'red', 'symbol': 'triangle-down', 'size': 10, 'line': {'color': 'darkred', 'width': 1}}
)